In [ ]:
# Test  AI Agent framework  AI A real estate agent 

In [53]:
import os , json, time , gc

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
import asyncio
import pandas as pd
import numpy as np
import torch


import nest_asyncio
nest_asyncio.apply() # for jupyter notebook

In [54]:
from crewai import Crew, Agent, Task, Process 
from crewai_tools import BraveSearchTool
import crewai
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any


In [55]:
class CFG:
    # search engine
    num_results = 3 # 5
    queryTime = "month" #"year"

    # Azure Opena Model
    llmModel1 = "gpt-4o-mini"
    llmModel2 = "gpt-4o"
    embedModel1 = "text-embedding-3-small" #1528 openai embedding model
    embedModel2 = "text-embedding-3-large"  # around 3000 dimension
    embedModel3 = "text-embedding-ada-002"
    embedModel4 = "azure-text-embedding-3-small"
    embedModel5 = "azure-text-embedding-3-large"

  
    
    openEmbedModel1 = "intfloat/multilingual-e5-small" # 512 dimension
    openEmbedModel2 = "sentence-transformers/distiluse-base-multilingual-cased-v1"
    openEmbedModel3 = "intfloat/multilingual-e5-large" # curent  # 1024 dimension 
    openEmbedModel4 = "Alibaba-NLP/gte-multilingual-base"
    openEmbedModel5 = "nomic-embed-text" # 768 dimension , support 8196 input 


    visionEmbedModel1 = "nomic-ai/nomic-embed-vision-v1.5"
    visionEmbedModel2 =  "openai/clip-vit-base-patch16" # openai vision embeding model 


    promptInjectModel1 = "protectai/deberta-v3-base-prompt-injection-v2"

    htmlMarkdownModel1 = "jinaai/reader-lm-0.5b"
    htmlMarkdownModel2 = "jinaai/reader-lm-1.5b"


    # Load locally Open source Embedding model 
    modelPath = "./model" 
    fullModelPath = modelPath + "/" + openEmbedModel3

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [57]:
from langchain_ollama import OllamaLLM, ChatOllama
from langchain_openai import ChatOpenAI

In [58]:
import ctypes
def clearMemory():
    for _ in range(5):
        torch.cuda.empty_cache()
        ctypes.CDLL("libc.so.6").malloc_trim(0)
        gc.collect()
        time.sleep(0.3)

In [59]:
query="Find me a 3 bedroom house in Hong Kong under $5 million"

In [60]:
# from crewai_tools import BraveSearchTool

# # Initialize the tool for internet searching capabilities
# tool = BraveSearchTool()

# SearxNG Web Search Engine

You can [check this link](https://docs.searxng.org/dev/search_api.html) for more informations about `Searx API` parameters.
#### Download SearxNG docker for local host setup web search Engine

In [66]:
import pprint
from langchain_community.utilities import SearxSearchWrapper

In [69]:
searchGeneralConfig = "!general"
searchImageConfig = "!images !bii !brimg !ddi !goi"
searchVideoConfig = "!videos !biv !brvid !ddv !gov"
searchNewConfig = "!news !gon !yhn"
searchMapConfig = "!map paris"
searchYouTubeConfig = "!yt"
searchSocialMedia = "!social_media !re !toot !mah !leco"

In [70]:
searxHost = "http://localhost:8080" #for langchain
searxHost2 = "http://localhost:8080/search" # for Searxng API directly
search = SearxSearchWrapper(searx_host=searxHost, k=5)

In [71]:
## Find URL web page for the query

def searchURL(q, k=3, qureyTime=CFG.queryTime, 
              engines=["duckduckgo", "qwant", "google", "brave"],
              categories = "general",
              querySuffix = None):
    results = search.results(q, 
                        num_results=k,
                        time_range = qureyTime,
                        engines= engines,
                        categories = categories,
                       query_suffix = querySuffix,
                       )
    
    return results

async def asyncSearchURL(q, k=3 , qureyTime=CFG.queryTime, 
                         engines=["duckduckgo", "qwant", "google", "brave"],
                         categories = "general",
                         querySuffix = None
                        ):
    startTime = time.time()
    results = await search.aresults(q, 
                        num_results=k,
                        time_range = qureyTime,
                        engines= engines,
                        categories = categories,
                        query_suffix = querySuffix,)
    print(f"Time take: {time.time() - startTime}")
    print(f"Query Categores : {categories}")
    return results
    

In [74]:
# %%time
queryTime = "month" #"year"#"month" #"year" #"day"
k = 5#5
currentQuery = "What is Hong Kong Weather Now?" #q2 #"Hong Kong Kwun Tong" #q3#tempQuery#q1
categories =  []#[""] #["science"]#["science", "image"]
query_suffix = None #searchMapConfig   # searchVideoConfig
urls = await asyncSearchURL(currentQuery, k=k, qureyTime=queryTime, categories=categories,  querySuffix= query_suffix)
print(currentQuery)
urls

Time take: 0.9366345405578613
Query Categores : []
What is Hong Kong Weather Now?


[{'snippet': '3 weeks ago - Stay informed and make the most of your day with our live weather updates from Hong Kong today. The temperature today in Hong Kong is expected to be typical for this time of year, with a forecast of 22 °C, closely aligning with the historical average for 6th of April.',
  'title': "Today's Weather in Hong Kong - Hourly Forecast and Conditions",
  'link': 'https://www.easeweather.com/asia/hong-kong/today',
  'engines': ['brave'],
  'category': 'general'},
 {'snippet': '2 weeks ago - Summer in Hong Kong is from late May to September. Weather: Temperatures commonly exceed 31 °C (88 °F).',
  'title': 'Best Times to Visit Hong Kong 2025 & Typhoon Season',
  'link': 'https://www.chinahighlights.com/hong-kong/weather.htm',
  'engines': ['brave'],
  'category': 'general'},
 {'snippet': '2 weeks ago - The Post breaks down what’s behind the dusty, dry weather that triggered a health warning.',
  'title': 'Explainer | Why is Hong Kong suddenly blanketed in haze and how

# Web scaping tool used Crawl4AI tool base on Playwrigth and Chrowmimum browswer


In [77]:
# Bros
# Test  playwright engine
from playwright.async_api import async_playwright

async def playwrightTestWeb():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True) # launch browser
        page = await browser.new_page()
        await page.goto('https://www.google.com')
        print(f"Title of the page: {await page.title()}")
        await page.screenshot(path='example.png')# save screenshot
        await browser.close()

In [ ]:
await playwrightTestWeb() # test the browser pylaywright engine

Title of the page: Google
